In [1]:
#import tensorflow_addons as tfa
#import keras
import time
#time.sleep(60*60*8)
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras import models, regularizers, layers, optimizers, losses, metrics
from tensorflow.keras.layers import Input, Concatenate, Conv2D, Flatten, Dense, Add, Dropout, Flatten, MaxPooling2D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping,ReduceLROnPlateau, TensorBoard
from tensorflow.keras.preprocessing.image import load_img, img_to_array, ImageDataGenerator,array_to_img
from keras import backend
import tensorflow.compat.v1.keras.backend as K

import numpy as np
from numpy import asarray
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix, precision_score, recall_score, classification_report, ConfusionMatrixDisplay, roc_auc_score, matthews_corrcoef,plot_roc_curve,roc_curve,average_precision_score
from sklearn.model_selection import train_test_split, KFold
from sklearn.utils import class_weight
from sklearn.calibration import calibration_curve
from sklearn.model_selection import StratifiedKFold

from PIL import Image
import seaborn as sns
sns.set_style("darkgrid")
from skimage import data, color, img_as_ubyte ,io
from skimage.feature import canny
from skimage.transform import hough_ellipse
from skimage.draw import ellipse_perimeter
from skimage.color import rgb2gray,rgba2rgb
import cv2

from tqdm import tqdm
from scipy.stats import mode
import imutils
from tensorflow.python.client import device_lib
import os
import random
import urllib.request
from vit_keras import vit
import csv
import random
def decode(a):
    if a =='A' or a =='2':
        return int(2)
    elif  a=='B' or a == '1':
        return int(1)
    elif a=='C' or a =='0':
        return int(0)
    else:
        return int(a)
def flatten_extend(matrix):
    flat_list = []
    for row in matrix:
         flat_list.extend(row)
    return flat_list

tf.keras.backend.clear_session()
os.environ["CUDA_VISIBLE_DEVICES"] = "1" 
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.compat.v1.Session(config=config)
# tf.compat.v1.set_random_seed(27)

/mnt/louisayu/nfs_share2/anaconda3/envs/tf/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.9.0 and strictly below 2.12.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.6.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  UserWarning,
2024-03-25 22:49:55.594802: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
T

In [2]:
preglist=os.listdir('/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/')

def split_list(lst, chunk_size):
    return [lst[i:i + chunk_size] for i in range(0, len(lst), chunk_size)]
print(len(preglist))
for i in range(10):
    print(len(split_list(preglist,len(preglist)//10)[i]))

test0=split_list(preglist,len(preglist)//10)[0]
train0=list(set(preglist)-set(test0))
train0.extend(split_list(preglist,len(preglist)//10)[10])
test1=split_list(preglist,len(preglist)//10)[1]
train1=list(set(preglist)-set(test1))
train1.extend(split_list(preglist,len(preglist)//10)[10])
test2=split_list(preglist,len(preglist)//10)[2]
train2=list(set(preglist)-set(test2))
train2.extend(split_list(preglist,len(preglist)//10)[10])
test3=split_list(preglist,len(preglist)//10)[3]
train3=list(set(preglist)-set(test3))
train3.extend(split_list(preglist,len(preglist)//10)[10])
test4=split_list(preglist,len(preglist)//10)[4]
train4=list(set(preglist)-set(test4))
train4.extend(split_list(preglist,len(preglist)//10)[10])
test5=split_list(preglist,len(preglist)//10)[5]
train5=list(set(preglist)-set(test5))
train5.extend(split_list(preglist,len(preglist)//10)[10])
test6=split_list(preglist,len(preglist)//10)[6]
train6=list(set(preglist)-set(test6))
train6.extend(split_list(preglist,len(preglist)//10)[10])
test7=split_list(preglist,len(preglist)//10)[7]
train7=list(set(preglist)-set(test7))
train7.extend(split_list(preglist,len(preglist)//10)[10])
test8=split_list(preglist,len(preglist)//10)[8]
train8=list(set(preglist)-set(test8))
train8.extend(split_list(preglist,len(preglist)//10)[10])
test9=split_list(preglist,len(preglist)//10)[9]
train9=list(set(preglist)-set(test9))
train9.extend(split_list(preglist,len(preglist)//10)[10])

1489
148
148
148
148
148
148
148
148
148
148


In [4]:
pixel=256
channels=3
NUM_CLASSES=2

def modelfro():
    vit_model = vit.vit_l16(
            image_size = pixel,
            activation = 'linear',
            pretrained = True,
            include_top = True,
            pretrained_top =False,)

    input_tensor = layers.Input(shape=(pixel,pixel,channels),name='vitinput')
    re=tf.keras.layers.Resizing(pixel,pixel,name='vitresize')(input_tensor)
    i1=layers.RandomRotation(factor=(-0.4, -0.2))(re)
    # i2=layers.RandomZoom(height_factor=-0.1, width_factor=-0.1)(i1)
    i3=layers.RandomFlip(mode="horizontal_and_vertical")(i1)
    bn=layers.BatchNormalization()(i3)
    conv_base = vit_model(bn)
    o1=layers.Dense(196,activation='elu',name='vitdense1')(conv_base)
    output=layers.Dropout(0.3)(o1)
    lb= tf.keras.models.Model(input_tensor ,output)
    
    input_g1 = layers.Input(shape=(1),name='G1')
    ig1=layers.Dense(1,activation='linear')(input_g1)
    mf1= tf.keras.models.Model(input_g1,ig1)
    
    input_g2 = layers.Input(shape=(1),name='G2')
    ig2=layers.Dense(1,activation='linear')(input_g2)
    mf2= tf.keras.models.Model(input_g2,ig2)
    
    input_g3 = layers.Input(shape=(1),name='G3')
    ig3=layers.Dense(1,activation='linear')(input_g3)
    mf3= tf.keras.models.Model(input_g3,ig3)

    x = Concatenate()([mf1.output,mf2.output,mf3.output,lb.output])  
    x = layers.Dense(32,activation='tanh')(x)

    output_tensor=layers.Dense(1,activation='sigmoid')(x)
    model=tf.keras.models.Model([mf1.input,mf2.input,mf3.input,lb.input],output_tensor)
    return model

def get_callbacks():
    checkpoint_filepath = '/mnt/louisayu/nfs_share2/embryo/model_weights/lb_fro/fro/ep{epoch:03d}_acc{accuracy:.4f}_val_loss{loss:.4f}.h5'
    return [
            EarlyStopping("val_accuracy", mode="max", patience = 13, verbose=1, restore_best_weights=True),
            
            ReduceLROnPlateau(monitor="val_loss", mode="min", factor=0.6, min_lr=1e-6, patience=5, verbose=1),
            
            ModelCheckpoint(checkpoint_filepath, monitor="val_loss", mode="min", save_weights_only=True, save_best_only=True)
           ]
def plot_roc_curve(fpr, tpr):
    plt.plot(fpr, tpr, color='orange', label='ROC')
    plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend()
    plt.show()
def get_f1(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val

def expect_f1(y_prob, thres):
    idxs = np.where(y_prob >= thres)[0]
    tp = y_prob[idxs].sum()
    fp = len(idxs) - tp
    idxs = np.where(y_prob < thres)[0]
    fn = y_prob[idxs].sum()
    return 2*tp / (2*tp + fp + fn)

def optimal_threshold(y_prob):
    y_prob = np.sort(y_prob)[::-1]
    f1s = [expect_f1(y_prob, p) for p in y_prob]
    thres = y_prob[np.argmax(f1s)]
    return thres, f1s


In [ ]:

train_image=[]
train_label=[]
G1=[]
G2=[]
G3=[]
#input training img and label
for f in train0:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        train_image.append(img)
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
        G1.append(int(p['G1'][i])-1)
        G2.append(decode(p['G2'][i]))
        G3.append(decode(p['G3'][i]))

a0= [x for x, y in list(enumerate(train_label)) if y ==0]
a1= [x for x, y in list(enumerate(train_label)) if y ==1]
print(len(a0),len(a1))   

#preprocess input
xxtrain = np.array(train_image,dtype='float32')
yytrain = np.array(train_label)
xxG1=np.array(G1,dtype='float32')
xxG2=np.array(G2,dtype='float32')
xxG3=np.array(G3,dtype='float32')
index = [i for i in range(len(xxtrain))]
np.random.seed(27)
np.random.shuffle(index)
xtrain = xxtrain[index]
ytrain = yytrain[index]
G11=xxG1[index]
G22=xxG2[index]
G33=xxG3[index]
x_train = np.reshape(xtrain,(len(xtrain),pixel,pixel,channels))
y_train = ytrain
G1x_train = np.reshape(G11,(len(G11),1))
G2x_train = np.reshape(G22,(len(G22),1))
G3x_train = np.reshape(G33,(len(G33),1))


#split train/val
index = [i for i in range(len(x_train))]
random.seed(27)
valindex=random.sample(index ,int(len(index)*0.15))
trainindex=list(set(index).difference(set(valindex)))
Xval=x_train[valindex]
Yval=y_train[valindex]
G1val=G1x_train[valindex]
G2val=G2x_train[valindex]
G3val=G3x_train[valindex]
Xtrain=x_train[trainindex]
Ytrain=y_train[trainindex]
G1train=G1x_train[trainindex]
G2train=G2x_train[trainindex]
G3train=G3x_train[trainindex]

#fit model
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)
model = modelfro()
opt = tf.keras.optimizers.Adam(learning_rate=0.00002)
model.compile(optimizer=opt,  loss=tf.keras.losses.binary_crossentropy, metrics =['accuracy', tf.keras.metrics.AUC(multi_label = False, name='aucc'),get_f1])
history=model.fit(x=[G1train,G2train,G3train,Xtrain], y= Ytrain, batch_size=32, validation_data=([G1val,G2val,G3val,Xval],Yval) , epochs=80, verbose=2, shuffle=True ,callbacks=get_callbacks(), class_weight=d_class_weights)
#input test img and label

y_true=Yval
y_scores=model.predict([G1val,G2val,G3val,Xval])
trues, preds = calibration_curve(y_true, y_scores, n_bins=5)
plt.plot(preds, trues, marker='o')
plt.xlabel("Posterior")
plt.ylabel("Probability being positive in each bin")
plt.xlim(([0, 1]))
plt.ylim(([0, 1]))
plt.title("Calibration plot")
thres2, f1s = optimal_threshold(y_scores)
print("Predicted Optimal Threshold is" ,thres2," with F1 score:",expect_f1(y_scores, thres2))

for f in test0:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    test_label=[]
    test_image=[]
    tG1=[]
    tG2=[]
    tG3=[]
    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        test_image.append(img)
        if p['LBOutc1'][i]==7:
            test_label.append(1)
        else:
            test_label.append(0)

        tG1.append(int(p['G1'][i])-1)
        tG2.append(decode(p['G2'][i]))
        tG3.append(decode(p['G3'][i]))

    xtest_arr = np.array(test_image,dtype='float32')
    ytest_arr = np.array(test_label)
    tG1_arr=np.array(tG1,dtype='float32')
    tG2_arr=np.array(tG2,dtype='float32')
    tG3_arr=np.array(tG3,dtype='float32')
    x_test = np.reshape(xtest_arr,(len(xtest_arr),pixel,pixel,channels))
    y_test = ytest_arr
    G1x_test = np.reshape(tG1_arr,(len(tG1_arr),1))
    G2x_test = np.reshape(tG2_arr,(len(tG2_arr),1))
    G3x_test = np.reshape(tG3_arr,(len(tG3_arr),1))
    y_pred=model.predict([G1x_test,G2x_test,G3x_test,x_test])
    y_pred_after_thre= np.where(y_pred>=thres2,1,0)
    with open("/mnt/louisayu/nfs_share2/embryo/code/final/fropredict/result/lbprob_imgfro/fold0_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j][0], y_pred_after_thre[j][0]]
            writer.writerow(content)
model.save('/mnt/louisayu/nfs_share2/embryo/model_weights/lb_fro/fro/model/'+str(thres2).replace('0.','final0_')+'.h5')


In [ ]:
train_image=[]
train_label=[]
G1=[]
G2=[]
G3=[]
#input training img and label
for f in train1:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        train_image.append(img)
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
        G1.append(int(p['G1'][i])-1)
        G2.append(decode(p['G2'][i]))
        G3.append(decode(p['G3'][i]))

a0= [x for x, y in list(enumerate(train_label)) if y ==0]
a1= [x for x, y in list(enumerate(train_label)) if y ==1]
print(len(a0),len(a1))   

#preprocess input
xxtrain = np.array(train_image,dtype='float32')
yytrain = np.array(train_label)
xxG1=np.array(G1,dtype='float32')
xxG2=np.array(G2,dtype='float32')
xxG3=np.array(G3,dtype='float32')
index = [i for i in range(len(xxtrain))]
np.random.seed(27)
np.random.shuffle(index)
xtrain = xxtrain[index]
ytrain = yytrain[index]
G11=xxG1[index]
G22=xxG2[index]
G33=xxG3[index]
x_train = np.reshape(xtrain,(len(xtrain),pixel,pixel,channels))
y_train = ytrain
G1x_train = np.reshape(G11,(len(G11),1))
G2x_train = np.reshape(G22,(len(G22),1))
G3x_train = np.reshape(G33,(len(G33),1))


#split train/val
index = [i for i in range(len(x_train))]
random.seed(27)
valindex=random.sample(index ,int(len(index)*0.15))
trainindex=list(set(index).difference(set(valindex)))
Xval=x_train[valindex]
Yval=y_train[valindex]
G1val=G1x_train[valindex]
G2val=G2x_train[valindex]
G3val=G3x_train[valindex]
Xtrain=x_train[trainindex]
Ytrain=y_train[trainindex]
G1train=G1x_train[trainindex]
G2train=G2x_train[trainindex]
G3train=G3x_train[trainindex]

#fit model
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)
model = modelfro()
opt = tf.keras.optimizers.Adam(learning_rate=0.00002)
model.compile(optimizer=opt,  loss=tf.keras.losses.binary_crossentropy, metrics =['accuracy', tf.keras.metrics.AUC(multi_label = False, name='aucc'),get_f1])
history=model.fit(x=[G1train,G2train,G3train,Xtrain], y= Ytrain, batch_size=32, validation_data=([G1val,G2val,G3val,Xval],Yval) , epochs=80, verbose=2, shuffle=True ,callbacks=get_callbacks(), class_weight=d_class_weights)


#calculate optimal threshold by f1
y_true=Yval
y_scores=model.predict([G1val,G2val,G3val,Xval])
trues, preds = calibration_curve(y_true, y_scores, n_bins=5)
plt.plot(preds, trues, marker='o')
plt.xlabel("Posterior")
plt.ylabel("Probability being positive in each bin")
plt.xlim(([0, 1]))
plt.ylim(([0, 1]))
plt.title("Calibration plot")
thres2, f1s = optimal_threshold(y_scores)
print("Predicted Optimal Threshold is" ,thres2," with F1 score:",expect_f1(y_scores, thres2))


#input test img and label

for f in test1:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    test_label=[]
    test_image=[]
    tG1=[]
    tG2=[]
    tG3=[]
    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        test_image.append(img)
        if p['LBOutc1'][i]==7:
            test_label.append(1)
        else:
            test_label.append(0)

        tG1.append(int(p['G1'][i])-1)
        tG2.append(decode(p['G2'][i]))
        tG3.append(decode(p['G3'][i]))

    xtest_arr = np.array(test_image,dtype='float32')
    ytest_arr = np.array(test_label)
    tG1_arr=np.array(tG1,dtype='float32')
    tG2_arr=np.array(tG2,dtype='float32')
    tG3_arr=np.array(tG3,dtype='float32')
    x_test = np.reshape(xtest_arr,(len(xtest_arr),pixel,pixel,channels))
    y_test = ytest_arr
    G1x_test = np.reshape(tG1_arr,(len(tG1_arr),1))
    G2x_test = np.reshape(tG2_arr,(len(tG2_arr),1))
    G3x_test = np.reshape(tG3_arr,(len(tG3_arr),1))
    y_pred=model.predict([G1x_test,G2x_test,G3x_test,x_test])
    y_pred_after_thre= np.where(y_pred>=thres2,1,0)
    with open("/mnt/louisayu/nfs_share2/embryo/code/final/fropredict/result/lbprob_imgfro/fold1_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j][0], y_pred_after_thre[j][0]]
            writer.writerow(content)
    
model.save('/mnt/louisayu/nfs_share2/embryo/model_weights/lb_fro/fro/model/'+str(thres2).replace('0.','final1_')+'.h5')


In [ ]:
train_image=[]
train_label=[]
G1=[]
G2=[]
G3=[]
#input training img and label
for f in train2:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        train_image.append(img)
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
        G1.append(int(p['G1'][i])-1)
        G2.append(decode(p['G2'][i]))
        G3.append(decode(p['G3'][i]))

a0= [x for x, y in list(enumerate(train_label)) if y ==0]
a1= [x for x, y in list(enumerate(train_label)) if y ==1]
print(len(a0),len(a1))   

#preprocess input
xxtrain = np.array(train_image,dtype='float32')
yytrain = np.array(train_label)
xxG1=np.array(G1,dtype='float32')
xxG2=np.array(G2,dtype='float32')
xxG3=np.array(G3,dtype='float32')
index = [i for i in range(len(xxtrain))]
np.random.seed(27)
np.random.shuffle(index)
xtrain = xxtrain[index]
ytrain = yytrain[index]
G11=xxG1[index]
G22=xxG2[index]
G33=xxG3[index]
x_train = np.reshape(xtrain,(len(xtrain),pixel,pixel,channels))
y_train = ytrain
G1x_train = np.reshape(G11,(len(G11),1))
G2x_train = np.reshape(G22,(len(G22),1))
G3x_train = np.reshape(G33,(len(G33),1))


#split train/val
index = [i for i in range(len(x_train))]
random.seed(27)
valindex=random.sample(index ,int(len(index)*0.15))
trainindex=list(set(index).difference(set(valindex)))
Xval=x_train[valindex]
Yval=y_train[valindex]
G1val=G1x_train[valindex]
G2val=G2x_train[valindex]
G3val=G3x_train[valindex]
Xtrain=x_train[trainindex]
Ytrain=y_train[trainindex]
G1train=G1x_train[trainindex]
G2train=G2x_train[trainindex]
G3train=G3x_train[trainindex]

#fit model
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)
model = modelfro()
opt = tf.keras.optimizers.Adam(learning_rate=0.00002)
model.compile(optimizer=opt,  loss=tf.keras.losses.binary_crossentropy, metrics =['accuracy', tf.keras.metrics.AUC(multi_label = False, name='aucc'),get_f1])
history=model.fit(x=[G1train,G2train,G3train,Xtrain], y= Ytrain, batch_size=32, validation_data=([G1val,G2val,G3val,Xval],Yval) , epochs=80, verbose=2, shuffle=True ,callbacks=get_callbacks(), class_weight=d_class_weights)


#calculate optimal threshold by f1
y_true=Yval
y_scores=model.predict([G1val,G2val,G3val,Xval])
trues, preds = calibration_curve(y_true, y_scores, n_bins=5)
plt.plot(preds, trues, marker='o')
plt.xlabel("Posterior")
plt.ylabel("Probability being positive in each bin")
plt.xlim(([0, 1]))
plt.ylim(([0, 1]))
plt.title("Calibration plot")
thres2, f1s = optimal_threshold(y_scores)
print("Predicted Optimal Threshold is" ,thres2," with F1 score:",expect_f1(y_scores, thres2))


#input test img and label
for f in test2:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    test_label=[]
    test_image=[]
    tG1=[]
    tG2=[]
    tG3=[]
    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        test_image.append(img)
        if p['LBOutc1'][i]==7:
            test_label.append(1)
        else:
            test_label.append(0)

        tG1.append(int(p['G1'][i])-1)
        tG2.append(decode(p['G2'][i]))
        tG3.append(decode(p['G3'][i]))

    xtest_arr = np.array(test_image,dtype='float32')
    ytest_arr = np.array(test_label)
    tG1_arr=np.array(tG1,dtype='float32')
    tG2_arr=np.array(tG2,dtype='float32')
    tG3_arr=np.array(tG3,dtype='float32')
    x_test = np.reshape(xtest_arr,(len(xtest_arr),pixel,pixel,channels))
    y_test = ytest_arr
    G1x_test = np.reshape(tG1_arr,(len(tG1_arr),1))
    G2x_test = np.reshape(tG2_arr,(len(tG2_arr),1))
    G3x_test = np.reshape(tG3_arr,(len(tG3_arr),1))
    y_pred=model.predict([G1x_test,G2x_test,G3x_test,x_test])
    y_pred_after_thre= np.where(y_pred>=thres2,1,0)
    with open("/mnt/louisayu/nfs_share2/embryo/code/final/fropredict/result/lbprob_imgfro/fold2_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j][0], y_pred_after_thre[j][0]]
            writer.writerow(content)
    
model.save('/mnt/louisayu/nfs_share2/embryo/model_weights/lb_fro/fro/model/'+str(thres2).replace('0.','final2_')+'.h5')


In [ ]:
train_image=[]
train_label=[]
G1=[]
G2=[]
G3=[]
#input training img and label
for f in train3:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        train_image.append(img)
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
        G1.append(int(p['G1'][i])-1)
        G2.append(decode(p['G2'][i]))
        G3.append(decode(p['G3'][i]))

a0= [x for x, y in list(enumerate(train_label)) if y ==0]
a1= [x for x, y in list(enumerate(train_label)) if y ==1]
print(len(a0),len(a1))   

#preprocess input
xxtrain = np.array(train_image,dtype='float32')
yytrain = np.array(train_label)
xxG1=np.array(G1,dtype='float32')
xxG2=np.array(G2,dtype='float32')
xxG3=np.array(G3,dtype='float32')
index = [i for i in range(len(xxtrain))]
np.random.seed(27)
np.random.shuffle(index)
xtrain = xxtrain[index]
ytrain = yytrain[index]
G11=xxG1[index]
G22=xxG2[index]
G33=xxG3[index]
x_train = np.reshape(xtrain,(len(xtrain),pixel,pixel,channels))
y_train = ytrain
G1x_train = np.reshape(G11,(len(G11),1))
G2x_train = np.reshape(G22,(len(G22),1))
G3x_train = np.reshape(G33,(len(G33),1))


#split train/val
index = [i for i in range(len(x_train))]
random.seed(27)
valindex=random.sample(index ,int(len(index)*0.15))
trainindex=list(set(index).difference(set(valindex)))
Xval=x_train[valindex]
Yval=y_train[valindex]
G1val=G1x_train[valindex]
G2val=G2x_train[valindex]
G3val=G3x_train[valindex]
Xtrain=x_train[trainindex]
Ytrain=y_train[trainindex]
G1train=G1x_train[trainindex]
G2train=G2x_train[trainindex]
G3train=G3x_train[trainindex]

#fit model
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)
model = modelfro()
opt = tf.keras.optimizers.Adam(learning_rate=0.00002)
model.compile(optimizer=opt,  loss=tf.keras.losses.binary_crossentropy, metrics =['accuracy', tf.keras.metrics.AUC(multi_label = False, name='aucc'),get_f1])
history=model.fit(x=[G1train,G2train,G3train,Xtrain], y= Ytrain, batch_size=32, validation_data=([G1val,G2val,G3val,Xval],Yval) , epochs=80, verbose=2, shuffle=True ,callbacks=get_callbacks(), class_weight=d_class_weights)

#calculate optimal threshold by f1
y_true=Yval
y_scores=model.predict([G1val,G2val,G3val,Xval])
trues, preds = calibration_curve(y_true, y_scores, n_bins=5)
plt.plot(preds, trues, marker='o')
plt.xlabel("Posterior")
plt.ylabel("Probability being positive in each bin")
plt.xlim(([0, 1]))
plt.ylim(([0, 1]))
plt.title("Calibration plot")
thres2, f1s = optimal_threshold(y_scores)
print("Predicted Optimal Threshold is" ,thres2," with F1 score:",expect_f1(y_scores, thres2))


#input test img and label
for f in test3:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    test_label=[]
    test_image=[]
    tG1=[]
    tG2=[]
    tG3=[]
    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        test_image.append(img)
        if p['LBOutc1'][i]==7:
            test_label.append(1)
        else:
            test_label.append(0)

        tG1.append(int(p['G1'][i])-1)
        tG2.append(decode(p['G2'][i]))
        tG3.append(decode(p['G3'][i]))

    xtest_arr = np.array(test_image,dtype='float32')
    ytest_arr = np.array(test_label)
    tG1_arr=np.array(tG1,dtype='float32')
    tG2_arr=np.array(tG2,dtype='float32')
    tG3_arr=np.array(tG3,dtype='float32')
    x_test = np.reshape(xtest_arr,(len(xtest_arr),pixel,pixel,channels))
    y_test = ytest_arr
    G1x_test = np.reshape(tG1_arr,(len(tG1_arr),1))
    G2x_test = np.reshape(tG2_arr,(len(tG2_arr),1))
    G3x_test = np.reshape(tG3_arr,(len(tG3_arr),1))
    y_pred=model.predict([G1x_test,G2x_test,G3x_test,x_test])
    y_pred_after_thre= np.where(y_pred>=thres2,1,0)
    with open("/mnt/louisayu/nfs_share2/embryo/code/final/fropredict/result/lbprob_imgfro/fold3_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j][0], y_pred_after_thre[j][0]]
            writer.writerow(content)
    
    
model.save('/mnt/louisayu/nfs_share2/embryo/model_weights/lb_fro/fro/model/'+str(thres2).replace('0.','final3_')+'.h5')


In [ ]:
train_image=[]
train_label=[]
G1=[]
G2=[]
G3=[]
#input training img and label
for f in train4:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        train_image.append(img)
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
        G1.append(int(p['G1'][i])-1)
        G2.append(decode(p['G2'][i]))
        G3.append(decode(p['G3'][i]))

a0= [x for x, y in list(enumerate(train_label)) if y ==0]
a1= [x for x, y in list(enumerate(train_label)) if y ==1]
print(len(a0),len(a1))   

#preprocess input
xxtrain = np.array(train_image,dtype='float32')
yytrain = np.array(train_label)
xxG1=np.array(G1,dtype='float32')
xxG2=np.array(G2,dtype='float32')
xxG3=np.array(G3,dtype='float32')
index = [i for i in range(len(xxtrain))]
np.random.seed(27)
np.random.shuffle(index)
xtrain = xxtrain[index]
ytrain = yytrain[index]
G11=xxG1[index]
G22=xxG2[index]
G33=xxG3[index]
x_train = np.reshape(xtrain,(len(xtrain),pixel,pixel,channels))
y_train = ytrain
G1x_train = np.reshape(G11,(len(G11),1))
G2x_train = np.reshape(G22,(len(G22),1))
G3x_train = np.reshape(G33,(len(G33),1))


#split train/val
index = [i for i in range(len(x_train))]
random.seed(27)
valindex=random.sample(index ,int(len(index)*0.15))
trainindex=list(set(index).difference(set(valindex)))
Xval=x_train[valindex]
Yval=y_train[valindex]
G1val=G1x_train[valindex]
G2val=G2x_train[valindex]
G3val=G3x_train[valindex]
Xtrain=x_train[trainindex]
Ytrain=y_train[trainindex]
G1train=G1x_train[trainindex]
G2train=G2x_train[trainindex]
G3train=G3x_train[trainindex]

#fit model
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)
model = modelfro()
opt = tf.keras.optimizers.Adam(learning_rate=0.00002)
model.compile(optimizer=opt,  loss=tf.keras.losses.binary_crossentropy, metrics =['accuracy', tf.keras.metrics.AUC(multi_label = False, name='aucc'),get_f1])
history=model.fit(x=[G1train,G2train,G3train,Xtrain], y= Ytrain, batch_size=32, validation_data=([G1val,G2val,G3val,Xval],Yval) , epochs=80, verbose=2, shuffle=True ,callbacks=get_callbacks(), class_weight=d_class_weights)


#calculate optimal threshold by f1
y_true=Yval
y_scores=model.predict([G1val,G2val,G3val,Xval])
trues, preds = calibration_curve(y_true, y_scores, n_bins=5)
plt.plot(preds, trues, marker='o')
plt.xlabel("Posterior")
plt.ylabel("Probability being positive in each bin")
plt.xlim(([0, 1]))
plt.ylim(([0, 1]))
plt.title("Calibration plot")
thres2, f1s = optimal_threshold(y_scores)
print("Predicted Optimal Threshold is" ,thres2," with F1 score:",expect_f1(y_scores, thres2))


#input test img and label

for f in test4:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    test_label=[]
    test_image=[]
    tG1=[]
    tG2=[]
    tG3=[]
    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        test_image.append(img)
        if p['LBOutc1'][i]==7:
            test_label.append(1)
        else:
            test_label.append(0)

        tG1.append(int(p['G1'][i])-1)
        tG2.append(decode(p['G2'][i]))
        tG3.append(decode(p['G3'][i]))

    xtest_arr = np.array(test_image,dtype='float32')
    ytest_arr = np.array(test_label)
    tG1_arr=np.array(tG1,dtype='float32')
    tG2_arr=np.array(tG2,dtype='float32')
    tG3_arr=np.array(tG3,dtype='float32')
    x_test = np.reshape(xtest_arr,(len(xtest_arr),pixel,pixel,channels))
    y_test = ytest_arr
    G1x_test = np.reshape(tG1_arr,(len(tG1_arr),1))
    G2x_test = np.reshape(tG2_arr,(len(tG2_arr),1))
    G3x_test = np.reshape(tG3_arr,(len(tG3_arr),1))
    y_pred=model.predict([G1x_test,G2x_test,G3x_test,x_test])
    y_pred_after_thre= np.where(y_pred>=thres2,1,0)
    with open("/mnt/louisayu/nfs_share2/embryo/code/final/fropredict/result/lbprob_imgfro/fold4_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j][0], y_pred_after_thre[j][0]]
            writer.writerow(content)
    
    
model.save('/mnt/louisayu/nfs_share2/embryo/model_weights/lb_fro/fro/model/'+str(thres2).replace('0.','final4_')+'.h5')


In [ ]:
train_image=[]
train_label=[]
G1=[]
G2=[]
G3=[]
#input training img and label
for f in train5:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        train_image.append(img)
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
        G1.append(int(p['G1'][i])-1)
        G2.append(decode(p['G2'][i]))
        G3.append(decode(p['G3'][i]))

a0= [x for x, y in list(enumerate(train_label)) if y ==0]
a1= [x for x, y in list(enumerate(train_label)) if y ==1]
print(len(a0),len(a1))   

#preprocess input
xxtrain = np.array(train_image,dtype='float32')
yytrain = np.array(train_label)
xxG1=np.array(G1,dtype='float32')
xxG2=np.array(G2,dtype='float32')
xxG3=np.array(G3,dtype='float32')
index = [i for i in range(len(xxtrain))]
np.random.seed(27)
np.random.shuffle(index)
xtrain = xxtrain[index]
ytrain = yytrain[index]
G11=xxG1[index]
G22=xxG2[index]
G33=xxG3[index]
x_train = np.reshape(xtrain,(len(xtrain),pixel,pixel,channels))
y_train = ytrain
G1x_train = np.reshape(G11,(len(G11),1))
G2x_train = np.reshape(G22,(len(G22),1))
G3x_train = np.reshape(G33,(len(G33),1))


#split train/val
index = [i for i in range(len(x_train))]
random.seed(27)
valindex=random.sample(index ,int(len(index)*0.15))
trainindex=list(set(index).difference(set(valindex)))
Xval=x_train[valindex]
Yval=y_train[valindex]
G1val=G1x_train[valindex]
G2val=G2x_train[valindex]
G3val=G3x_train[valindex]
Xtrain=x_train[trainindex]
Ytrain=y_train[trainindex]
G1train=G1x_train[trainindex]
G2train=G2x_train[trainindex]
G3train=G3x_train[trainindex]

#fit model
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)
model = modelfro()
opt = tf.keras.optimizers.Adam(learning_rate=0.00002)
model.compile(optimizer=opt,  loss=tf.keras.losses.binary_crossentropy, metrics =['accuracy', tf.keras.metrics.AUC(multi_label = False, name='aucc'),get_f1])
history=model.fit(x=[G1train,G2train,G3train,Xtrain], y= Ytrain, batch_size=32, validation_data=([G1val,G2val,G3val,Xval],Yval) , epochs=80, verbose=2, shuffle=True ,callbacks=get_callbacks(), class_weight=d_class_weights)

#calculate optimal threshold by f1
y_true=Yval
y_scores=model.predict([G1val,G2val,G3val,Xval])
trues, preds = calibration_curve(y_true, y_scores, n_bins=5)
plt.plot(preds, trues, marker='o')
plt.xlabel("Posterior")
plt.ylabel("Probability being positive in each bin")
plt.xlim(([0, 1]))
plt.ylim(([0, 1]))
plt.title("Calibration plot")
thres2, f1s = optimal_threshold(y_scores)
print("Predicted Optimal Threshold is" ,thres2," with F1 score:",expect_f1(y_scores, thres2))


#input test img and label


for f in test5:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    test_label=[]
    test_image=[]
    tG1=[]
    tG2=[]
    tG3=[]
    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        test_image.append(img)
        if p['LBOutc1'][i]==7:
            test_label.append(1)
        else:
            test_label.append(0)

        tG1.append(int(p['G1'][i])-1)
        tG2.append(decode(p['G2'][i]))
        tG3.append(decode(p['G3'][i]))

    xtest_arr = np.array(test_image,dtype='float32')
    ytest_arr = np.array(test_label)
    tG1_arr=np.array(tG1,dtype='float32')
    tG2_arr=np.array(tG2,dtype='float32')
    tG3_arr=np.array(tG3,dtype='float32')
    x_test = np.reshape(xtest_arr,(len(xtest_arr),pixel,pixel,channels))
    y_test = ytest_arr
    G1x_test = np.reshape(tG1_arr,(len(tG1_arr),1))
    G2x_test = np.reshape(tG2_arr,(len(tG2_arr),1))
    G3x_test = np.reshape(tG3_arr,(len(tG3_arr),1))
    y_pred=model.predict([G1x_test,G2x_test,G3x_test,x_test])
    y_pred_after_thre= np.where(y_pred>=thres2,1,0)
    with open("/mnt/louisayu/nfs_share2/embryo/code/final/fropredict/result/lbprob_imgfro/fold5_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j][0], y_pred_after_thre[j][0]]
            writer.writerow(content)
    
    
model.save('/mnt/louisayu/nfs_share2/embryo/model_weights/lb_fro/fro/model/'+str(thres2).replace('0.','final5_')+'.h5')


In [ ]:
train_image=[]
train_label=[]
G1=[]
G2=[]
G3=[]
#input training img and label
for f in train6:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        train_image.append(img)
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
        G1.append(int(p['G1'][i])-1)
        G2.append(decode(p['G2'][i]))
        G3.append(decode(p['G3'][i]))

a0= [x for x, y in list(enumerate(train_label)) if y ==0]
a1= [x for x, y in list(enumerate(train_label)) if y ==1]
print(len(a0),len(a1))   

#preprocess input
xxtrain = np.array(train_image,dtype='float32')
yytrain = np.array(train_label)
xxG1=np.array(G1,dtype='float32')
xxG2=np.array(G2,dtype='float32')
xxG3=np.array(G3,dtype='float32')
index = [i for i in range(len(xxtrain))]
np.random.seed(27)
np.random.shuffle(index)
xtrain = xxtrain[index]
ytrain = yytrain[index]
G11=xxG1[index]
G22=xxG2[index]
G33=xxG3[index]
x_train = np.reshape(xtrain,(len(xtrain),pixel,pixel,channels))
y_train = ytrain
G1x_train = np.reshape(G11,(len(G11),1))
G2x_train = np.reshape(G22,(len(G22),1))
G3x_train = np.reshape(G33,(len(G33),1))


#split train/val
index = [i for i in range(len(x_train))]
random.seed(27)
valindex=random.sample(index ,int(len(index)*0.15))
trainindex=list(set(index).difference(set(valindex)))
Xval=x_train[valindex]
Yval=y_train[valindex]
G1val=G1x_train[valindex]
G2val=G2x_train[valindex]
G3val=G3x_train[valindex]
Xtrain=x_train[trainindex]
Ytrain=y_train[trainindex]
G1train=G1x_train[trainindex]
G2train=G2x_train[trainindex]
G3train=G3x_train[trainindex]

#fit model
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)
model = modelfro()
opt = tf.keras.optimizers.Adam(learning_rate=0.00002)
model.compile(optimizer=opt,  loss=tf.keras.losses.binary_crossentropy, metrics =['accuracy', tf.keras.metrics.AUC(multi_label = False, name='aucc'),get_f1])
history=model.fit(x=[G1train,G2train,G3train,Xtrain], y= Ytrain, batch_size=32, validation_data=([G1val,G2val,G3val,Xval],Yval) , epochs=80, verbose=2, shuffle=True ,callbacks=get_callbacks(), class_weight=d_class_weights)

#calculate optimal threshold by f1
y_true=Yval
y_scores=model.predict([G1val,G2val,G3val,Xval])
trues, preds = calibration_curve(y_true, y_scores, n_bins=5)
plt.plot(preds, trues, marker='o')
plt.xlabel("Posterior")
plt.ylabel("Probability being positive in each bin")
plt.xlim(([0, 1]))
plt.ylim(([0, 1]))
plt.title("Calibration plot")
thres2, f1s = optimal_threshold(y_scores)
print("Predicted Optimal Threshold is" ,thres2," with F1 score:",expect_f1(y_scores, thres2))


#input test img and label

for f in test6:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    test_label=[]
    test_image=[]
    tG1=[]
    tG2=[]
    tG3=[]
    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        test_image.append(img)
        if p['LBOutc1'][i]==7:
            test_label.append(1)
        else:
            test_label.append(0)

        tG1.append(int(p['G1'][i])-1)
        tG2.append(decode(p['G2'][i]))
        tG3.append(decode(p['G3'][i]))

    xtest_arr = np.array(test_image,dtype='float32')
    ytest_arr = np.array(test_label)
    tG1_arr=np.array(tG1,dtype='float32')
    tG2_arr=np.array(tG2,dtype='float32')
    tG3_arr=np.array(tG3,dtype='float32')
    x_test = np.reshape(xtest_arr,(len(xtest_arr),pixel,pixel,channels))
    y_test = ytest_arr
    G1x_test = np.reshape(tG1_arr,(len(tG1_arr),1))
    G2x_test = np.reshape(tG2_arr,(len(tG2_arr),1))
    G3x_test = np.reshape(tG3_arr,(len(tG3_arr),1))
    y_pred=model.predict([G1x_test,G2x_test,G3x_test,x_test])
    y_pred_after_thre= np.where(y_pred>=thres2,1,0)
    with open("/mnt/louisayu/nfs_share2/embryo/code/final/fropredict/result/lbprob_imgfro/fold6_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j][0], y_pred_after_thre[j][0]]
            writer.writerow(content)
    
    
model.save('/mnt/louisayu/nfs_share2/embryo/model_weights/lb_fro/fro/model/'+str(thres2).replace('0.','final6_')+'.h5')


In [ ]:
train_image=[]
train_label=[]
G1=[]
G2=[]
G3=[]
#input training img and label
for f in train7:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        train_image.append(img)
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
        G1.append(int(p['G1'][i])-1)
        G2.append(decode(p['G2'][i]))
        G3.append(decode(p['G3'][i]))

a0= [x for x, y in list(enumerate(train_label)) if y ==0]
a1= [x for x, y in list(enumerate(train_label)) if y ==1]
print(len(a0),len(a1))   

#preprocess input
xxtrain = np.array(train_image,dtype='float32')
yytrain = np.array(train_label)
xxG1=np.array(G1,dtype='float32')
xxG2=np.array(G2,dtype='float32')
xxG3=np.array(G3,dtype='float32')
index = [i for i in range(len(xxtrain))]
np.random.seed(27)
np.random.shuffle(index)
xtrain = xxtrain[index]
ytrain = yytrain[index]
G11=xxG1[index]
G22=xxG2[index]
G33=xxG3[index]
x_train = np.reshape(xtrain,(len(xtrain),pixel,pixel,channels))
y_train = ytrain
G1x_train = np.reshape(G11,(len(G11),1))
G2x_train = np.reshape(G22,(len(G22),1))
G3x_train = np.reshape(G33,(len(G33),1))


#split train/val
index = [i for i in range(len(x_train))]
random.seed(27)
valindex=random.sample(index ,int(len(index)*0.15))
trainindex=list(set(index).difference(set(valindex)))
Xval=x_train[valindex]
Yval=y_train[valindex]
G1val=G1x_train[valindex]
G2val=G2x_train[valindex]
G3val=G3x_train[valindex]
Xtrain=x_train[trainindex]
Ytrain=y_train[trainindex]
G1train=G1x_train[trainindex]
G2train=G2x_train[trainindex]
G3train=G3x_train[trainindex]

#fit model
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)
model = modelfro()
opt = tf.keras.optimizers.Adam(learning_rate=0.00002)
model.compile(optimizer=opt,  loss=tf.keras.losses.binary_crossentropy, metrics =['accuracy', tf.keras.metrics.AUC(multi_label = False, name='aucc'),get_f1])
history=model.fit(x=[G1train,G2train,G3train,Xtrain], y= Ytrain, batch_size=32, validation_data=([G1val,G2val,G3val,Xval],Yval) , epochs=80, verbose=2, shuffle=True ,callbacks=get_callbacks(), class_weight=d_class_weights)

#calculate optimal threshold by f1
y_true=Yval
y_scores=model.predict([G1val,G2val,G3val,Xval])
trues, preds = calibration_curve(y_true, y_scores, n_bins=5)
plt.plot(preds, trues, marker='o')
plt.xlabel("Posterior")
plt.ylabel("Probability being positive in each bin")
plt.xlim(([0, 1]))
plt.ylim(([0, 1]))
plt.title("Calibration plot")
thres2, f1s = optimal_threshold(y_scores)
print("Predicted Optimal Threshold is" ,thres2," with F1 score:",expect_f1(y_scores, thres2))


#input test img and label
for f in test7:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    test_label=[]
    test_image=[]
    tG1=[]
    tG2=[]
    tG3=[]
    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        test_image.append(img)
        if p['LBOutc1'][i]==7:
            test_label.append(1)
        else:
            test_label.append(0)

        tG1.append(int(p['G1'][i])-1)
        tG2.append(decode(p['G2'][i]))
        tG3.append(decode(p['G3'][i]))

    xtest_arr = np.array(test_image,dtype='float32')
    ytest_arr = np.array(test_label)
    tG1_arr=np.array(tG1,dtype='float32')
    tG2_arr=np.array(tG2,dtype='float32')
    tG3_arr=np.array(tG3,dtype='float32')
    x_test = np.reshape(xtest_arr,(len(xtest_arr),pixel,pixel,channels))
    y_test = ytest_arr
    G1x_test = np.reshape(tG1_arr,(len(tG1_arr),1))
    G2x_test = np.reshape(tG2_arr,(len(tG2_arr),1))
    G3x_test = np.reshape(tG3_arr,(len(tG3_arr),1))
    y_pred=model.predict([G1x_test,G2x_test,G3x_test,x_test])
    y_pred_after_thre= np.where(y_pred>=thres2,1,0)
    with open("/mnt/louisayu/nfs_share2/embryo/code/final/fropredict/result/lbprob_imgfro/fold7_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j][0], y_pred_after_thre[j][0]]
            writer.writerow(content)
    
            
model.save('/mnt/louisayu/nfs_share2/embryo/model_weights/lb_fro/fro/model/'+str(thres2).replace('0.','final7_')+'.h5')


In [ ]:
train_image=[]
train_label=[]
G1=[]
G2=[]
G3=[]
#input training img and label
for f in train8:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        train_image.append(img)
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
        G1.append(int(p['G1'][i])-1)
        G2.append(decode(p['G2'][i]))
        G3.append(decode(p['G3'][i]))

a0= [x for x, y in list(enumerate(train_label)) if y ==0]
a1= [x for x, y in list(enumerate(train_label)) if y ==1]
print(len(a0),len(a1))   

#preprocess input
xxtrain = np.array(train_image,dtype='float32')
yytrain = np.array(train_label)
xxG1=np.array(G1,dtype='float32')
xxG2=np.array(G2,dtype='float32')
xxG3=np.array(G3,dtype='float32')
index = [i for i in range(len(xxtrain))]
np.random.seed(27)
np.random.shuffle(index)
xtrain = xxtrain[index]
ytrain = yytrain[index]
G11=xxG1[index]
G22=xxG2[index]
G33=xxG3[index]
x_train = np.reshape(xtrain,(len(xtrain),pixel,pixel,channels))
y_train = ytrain
G1x_train = np.reshape(G11,(len(G11),1))
G2x_train = np.reshape(G22,(len(G22),1))
G3x_train = np.reshape(G33,(len(G33),1))


#split train/val
index = [i for i in range(len(x_train))]
random.seed(27)
valindex=random.sample(index ,int(len(index)*0.15))
trainindex=list(set(index).difference(set(valindex)))
Xval=x_train[valindex]
Yval=y_train[valindex]
G1val=G1x_train[valindex]
G2val=G2x_train[valindex]
G3val=G3x_train[valindex]
Xtrain=x_train[trainindex]
Ytrain=y_train[trainindex]
G1train=G1x_train[trainindex]
G2train=G2x_train[trainindex]
G3train=G3x_train[trainindex]

#fit model
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)
model = modelfro()
opt = tf.keras.optimizers.Adam(learning_rate=0.00002)
model.compile(optimizer=opt,  loss=tf.keras.losses.binary_crossentropy, metrics =['accuracy', tf.keras.metrics.AUC(multi_label = False, name='aucc'),get_f1])

#calculate optimal threshold by f1
y_true=Yval
y_scores=model.predict([G1val,G2val,G3val,Xval])
trues, preds = calibration_curve(y_true, y_scores, n_bins=5)
plt.plot(preds, trues, marker='o')
plt.xlabel("Posterior")
plt.ylabel("Probability being positive in each bin")
plt.xlim(([0, 1]))
plt.ylim(([0, 1]))
plt.title("Calibration plot")
thres2, f1s = optimal_threshold(y_scores)
print("Predicted Optimal Threshold is" ,thres2," with F1 score:",expect_f1(y_scores, thres2))


#input test img and label

for f in test8:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    test_label=[]
    test_image=[]
    tG1=[]
    tG2=[]
    tG3=[]
    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        test_image.append(img)
        if p['LBOutc1'][i]==7:
            test_label.append(1)
        else:
            test_label.append(0)

        tG1.append(int(p['G1'][i])-1)
        tG2.append(decode(p['G2'][i]))
        tG3.append(decode(p['G3'][i]))

    xtest_arr = np.array(test_image,dtype='float32')
    ytest_arr = np.array(test_label)
    tG1_arr=np.array(tG1,dtype='float32')
    tG2_arr=np.array(tG2,dtype='float32')
    tG3_arr=np.array(tG3,dtype='float32')
    x_test = np.reshape(xtest_arr,(len(xtest_arr),pixel,pixel,channels))
    y_test = ytest_arr
    G1x_test = np.reshape(tG1_arr,(len(tG1_arr),1))
    G2x_test = np.reshape(tG2_arr,(len(tG2_arr),1))
    G3x_test = np.reshape(tG3_arr,(len(tG3_arr),1))
    y_pred=model.predict([G1x_test,G2x_test,G3x_test,x_test])
    y_pred_after_thre= np.where(y_pred>=thres2,1,0)
    with open("/mnt/louisayu/nfs_share2/embryo/code/final/fropredict/result/lbprob_imgfro/fold8_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j][0], y_pred_after_thre[j][0]]
            writer.writerow(content)
    
            
model.save('/mnt/louisayu/nfs_share2/embryo/model_weights/lb_fro/fro/model/'+str(thres2).replace('0.','final8_')+'.h5')


In [ ]:
train_image=[]
train_label=[]
G1=[]
G2=[]
G3=[]
#input training img and label
for f in train9:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')

    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        train_image.append(img)
        if p['LBOutc1'][i]==7:
            train_label.append(1)
        else:
            train_label.append(0)
        G1.append(int(p['G1'][i])-1)
        G2.append(decode(p['G2'][i]))
        G3.append(decode(p['G3'][i]))

a0= [x for x, y in list(enumerate(train_label)) if y ==0]
a1= [x for x, y in list(enumerate(train_label)) if y ==1]
print(len(a0),len(a1))   

#preprocess input
xxtrain = np.array(train_image,dtype='float32')
yytrain = np.array(train_label)
xxG1=np.array(G1,dtype='float32')
xxG2=np.array(G2,dtype='float32')
xxG3=np.array(G3,dtype='float32')
index = [i for i in range(len(xxtrain))]
np.random.seed(27)
np.random.shuffle(index)
xtrain = xxtrain[index]
ytrain = yytrain[index]
G11=xxG1[index]
G22=xxG2[index]
G33=xxG3[index]
x_train = np.reshape(xtrain,(len(xtrain),pixel,pixel,channels))
y_train = ytrain
G1x_train = np.reshape(G11,(len(G11),1))
G2x_train = np.reshape(G22,(len(G22),1))
G3x_train = np.reshape(G33,(len(G33),1))


#split train/val
index = [i for i in range(len(x_train))]
random.seed(27)
valindex=random.sample(index ,int(len(index)*0.15))
trainindex=list(set(index).difference(set(valindex)))
Xval=x_train[valindex]
Yval=y_train[valindex]
G1val=G1x_train[valindex]
G2val=G2x_train[valindex]
G3val=G3x_train[valindex]
Xtrain=x_train[trainindex]
Ytrain=y_train[trainindex]
G1train=G1x_train[trainindex]
G2train=G2x_train[trainindex]
G3train=G3x_train[trainindex]

#fit model
class_weights = class_weight.compute_class_weight(class_weight='balanced', classes=np.unique(y_train), y=y_train)
d_class_weights = dict(enumerate(class_weights))
print(d_class_weights)
model = modelfro()
opt = tf.keras.optimizers.Adam(learning_rate=0.00002)
model.compile(optimizer=opt,  loss=tf.keras.losses.binary_crossentropy, metrics =['accuracy', tf.keras.metrics.AUC(multi_label = False, name='aucc'),get_f1])
history=model.fit(x=[G1train,G2train,G3train,Xtrain], y= Ytrain, batch_size=32, validation_data=([G1val,G2val,G3val,Xval],Yval) , epochs=80, verbose=2, shuffle=True ,callbacks=get_callbacks(), class_weight=d_class_weights)

#calculate optimal threshold by f1
y_true=Yval
y_scores=model.predict([G1val,G2val,G3val,Xval])
trues, preds = calibration_curve(y_true, y_scores, n_bins=5)
plt.plot(preds, trues, marker='o')
plt.xlabel("Posterior")
plt.ylabel("Probability being positive in each bin")
plt.xlim(([0, 1]))
plt.ylim(([0, 1]))
plt.title("Calibration plot")
thres2, f1s = optimal_threshold(y_scores)
print("Predicted Optimal Threshold is" ,thres2," with F1 score:",expect_f1(y_scores, thres2))


#input test img and label

for f in test9:
    p=pd.read_excel(r'/mnt/louisayu/nfs_share2/embryo/code/em_aug/ACC/'+f, engine='openpyxl')
    test_label=[]
    test_image=[]
    tG1=[]
    tG2=[]
    tG3=[]
    for i in range(p.shape[0]):
        filename= "/mnt/louisayu/nfs_share2/embryo/preprocess_rec_1018/"+p['image'][i]
        im = cv2.imread(filename)#,cv2.IMREAD_GRAYSCALE)
        im = cv2.resize(im,(pixel,pixel))
        im = cv2.normalize(im, None, 0, 255, norm_type=cv2.NORM_MINMAX)
        img = img_to_array(im)
        test_image.append(img)
        if p['LBOutc1'][i]==7:
            test_label.append(1)
        else:
            test_label.append(0)

        tG1.append(int(p['G1'][i])-1)
        tG2.append(decode(p['G2'][i]))
        tG3.append(decode(p['G3'][i]))

    xtest_arr = np.array(test_image,dtype='float32')
    ytest_arr = np.array(test_label)
    tG1_arr=np.array(tG1,dtype='float32')
    tG2_arr=np.array(tG2,dtype='float32')
    tG3_arr=np.array(tG3,dtype='float32')
    x_test = np.reshape(xtest_arr,(len(xtest_arr),pixel,pixel,channels))
    y_test = ytest_arr
    G1x_test = np.reshape(tG1_arr,(len(tG1_arr),1))
    G2x_test = np.reshape(tG2_arr,(len(tG2_arr),1))
    G3x_test = np.reshape(tG3_arr,(len(tG3_arr),1))
    y_pred=model.predict([G1x_test,G2x_test,G3x_test,x_test])
    y_pred_after_thre= np.where(y_pred>=thres2,1,0)
    with open("/mnt/louisayu/nfs_share2/embryo/code/final/fropredict/result/lbprob_imgfro/fold9_"+f, "w", newline='') as csvfile:
        
        writer = csv.writer(csvfile)
        writer.writerow(['true','pred','pred_thre'])
        for j in range(len(y_test)):
            content = [y_test[j], y_pred[j][0], y_pred_after_thre[j][0]]
            writer.writerow(content)
    
    
model.save('/mnt/louisayu/nfs_share2/embryo/model_weights/lb_fro/fro/model/'+str(thres2).replace('0.','final9_')+'.h5')


In [16]:
resultdir="/mnt/louisayu/nfs_share2/embryo/code/final/fropredict/result/lbprob_imgfro/"
for i in range(0,10):
    ranking=0
    multisuccess_patients=0
    pati_list=[j for j in os.listdir(resultdir) if 'fold'+str(i) in j]
    for k in pati_list:
        p=pd.read_csv(resultdir+k)
        if p.shape[0]>1 and 1 in list(p['true']):
            multisuccess_patients=multisuccess_patients+1
            if p['true'][np.argmax(p['pred'])]==1:
                ranking=ranking+1
            else:
                continue
    print("top1accuracy for ",i,"-th fold:", ranking/multisuccess_patients)

0
148
top1accuracy for  0 -th fold: 0.8095238095238095
1
148
top1accuracy for  1 -th fold: 0.6666666666666666
2
148
top1accuracy for  2 -th fold: 0.5526315789473685
3
148
top1accuracy for  3 -th fold: 0.5833333333333334
4
148
top1accuracy for  4 -th fold: 0.5714285714285714
5
148
top1accuracy for  5 -th fold: 0.5833333333333334
6
148
top1accuracy for  6 -th fold: 0.625
7
148
top1accuracy for  7 -th fold: 0.5172413793103449
8
148
top1accuracy for  8 -th fold: 0.5769230769230769
9
148
top1accuracy for  9 -th fold: 0.5238095238095238


In [19]:
resultdir="/mnt/louisayu/nfs_share2/embryo/code/final/fropredict/result/lbprob_imgfro/"
for i in range(0,10):
    multisuccess_truepred=0
    multifail_truepred=0
    singlesuccess_truepred=0
    singfail_truepred=0
    multisuccess_patients=0
    multifail_patients=0
    singlesuccess_patients=0
    singfail_patients=0
    pati_list=[j for j in os.listdir(resultdir) if 'fold'+str(i) in j]
    for k in pati_list:
        p=pd.read_csv(resultdir+k)
        if p.shape[0]>1 and 1 in list(p['true']):
            multisuccess_patients=multisuccess_patients+1
            if p['true'][np.argmax(p['pred'])]==1 and p['pred_thre'][np.argmax(p['pred'])]==1:
                multisuccess_truepred=multisuccess_truepred+1
            else:
                continue
        elif p.shape[0]>1 and 1 not in list(p['true']):
            multifail_patients=multifail_patients+1
            if p['true'][np.argmax(p['pred'])]==0 and p['pred_thre'][np.argmax(p['pred'])]==0:
                multifail_truepred=multifail_truepred+1     
            else:
                continue
        elif p.shape[0]==1 and 1 not in list(p['true']):
            singfail_patients=singfail_patients+1
            if p['true'][np.argmax(p['pred'])]==0 and p['pred_thre'][np.argmax(p['pred'])]==0:
                singfail_truepred=singfail_truepred+1     
            else:
                continue
        elif p.shape[0]==1 and 1 in list(p['true']):
            singlesuccess_patients=singlesuccess_patients+1
            if p['true'][np.argmax(p['pred'])]==1 and p['pred_thre'][np.argmax(p['pred'])]==1:
                singlesuccess_truepred=singlesuccess_truepred+1     
            else:
                continue
        
        
        
    print("top1accuracy for ",i,"-th fold:", (multisuccess_truepred+multifail_truepred+singlesuccess_truepred+singfail_truepred)/(multisuccess_patients+multifail_patients+singlesuccess_patients+singfail_patients))

top1accuracy for  0 -th fold: 0.6418918918918919
top1accuracy for  1 -th fold: 0.5202702702702703
top1accuracy for  2 -th fold: 0.5202702702702703
top1accuracy for  3 -th fold: 0.5067567567567568
top1accuracy for  4 -th fold: 0.5608108108108109
top1accuracy for  5 -th fold: 0.5878378378378378
top1accuracy for  6 -th fold: 0.6081081081081081
top1accuracy for  7 -th fold: 0.6013513513513513
top1accuracy for  8 -th fold: 0.5337837837837838
top1accuracy for  9 -th fold: 0.6081081081081081
